## Inpsecting the annotations.jsonl file

add hyperparameter search!

In [1]:
import sklearn
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
import scipy.stats
import eli5
import json

with open("../annotations.jsonl") as jsonl_file:
    lines = jsonl_file.readlines()
annot = [json.loads(line) for line in lines]
print("instances:\n{}".format(len(annot)))
keys = [key for key in annot[0].keys()]
print("\nall keys:\n{}".format(keys))
key_keys = ["text", "spans", "tokens"]
print("\nimportant keys:\n{}".format(key_keys))
print("\nexample text:\n{}".format(annot[0]["text"]))
n_examples = 5
print("\n{} example spans:".format(n_examples))
for span in annot[0]["spans"][:n_examples]:
    print("{}".format(span))
print("\n{} example tokens:".format(n_examples))
for token in annot[0]["tokens"][:n_examples]:
    print("{}".format(token))

instances:
140

all keys:
['text', 'meta', '_input_hash', '_task_hash', 'spans', 'tokens', '_view_id', 'answer', '_timestamp']

important keys:
['text', 'spans', 'tokens']

example text:
DORNBIRN In der Schulgasse in Dornbirn hat eine 71,93 Quadratmeter große Wohnung für einen Quadratmeterpreis von 5533,71 Euro den Besitzer gewechselt. Dieser beinhaltet auch einen Pkw-Abstellplatz. Käufer der Wohnung mit 9,86 Quadratmetern Terrasse ist die ValLiLean Beteiligungs- und Immobilienverwaltungs GmbH. Beim Verkäufer handelt es sich um die Karrenblick Projekt GmbH.  Der Kaufpreis liegt bei 398.040 Euro. Unterzeichnet wurde der Kaufvertrag am 18. September. Die Verbücherung datiert mit Oktober 2020.

5 example spans:
{'text': 'DORNBIRN', 'start': 0, 'end': 8, 'pattern': 2069086582, 'token_start': 0, 'token_end': 0, 'label': 'ORT'}
{'start': 16, 'end': 26, 'token_start': 3, 'token_end': 3, 'label': 'STRASSE'}
{'text': 'Dornbirn', 'start': 30, 'end': 38, 'pattern': 2069086582, 'token_start': 5, '

In [2]:
def getLabel(dictList, idx):
    result = "O"
    for dict_i in dictList:
        idx_0, idx_1 = dict_i['start'], dict_i['end']
        if (idx_0<=idx) and (idx<=idx_1):
            result = dict_i["label"]
    return result 

myDictList = [
    {'start':0, 'end':3, 'label': 'ORT'},
    {'start':5, 'end':8, 'label': 'ORT2'},
    {'start':10, 'end':13, 'label': 'ORT3'}
]

for i in range(16):
    pos = i-1
    print("label for position {}:\t{}".format(pos, getLabel(myDictList, i-1)))

label for position -1:	O
label for position 0:	ORT
label for position 1:	ORT
label for position 2:	ORT
label for position 3:	ORT
label for position 4:	O
label for position 5:	ORT2
label for position 6:	ORT2
label for position 7:	ORT2
label for position 8:	ORT2
label for position 9:	O
label for position 10:	ORT3
label for position 11:	ORT3
label for position 12:	ORT3
label for position 13:	ORT3
label for position 14:	O


In [3]:
check = True
for j in range(len(annot)): # loop over instances
    a = annot[j]            # instance j
    spans = a['spans']      # list of annotation dicts
    toks = a['tokens']      # list of token dicts
    for i in range(len(toks)):                                 # loop over token dicts
        toks[i]['label'] = getLabel(spans, toks[i]['start'])   # assign label from span (if exists, otherwise "O")
        if toks[i]['label'] != "O":                            # if the token represents an entity ...
            if i==0:
                toks[i]['label'] = "B-"+toks[i]['label']       # ... and is the first in the text => "B-" + label
            else:                                              # not first token in text:
                if (toks[i]['label']==toks[i-1]['label'][2:]):
                    toks[i]['label'] = "I-"+toks[i]['label']   # > but same label as previous token => "I-" + label
                else:
                    toks[i]['label'] = "B-"+toks[i]['label']   # > but first token of an entity => "B-" + label
    annot[j]['tokens'] = toks

words_n = 3
for i in range(2):
    print("Token dictionaries for the last {} words of instance {}".format(words_n, i))
    ann = annot[i]
    for tok in ann["tokens"][:words_n]:
        print(tok)
# O => trivial class (no entity)
# B => Entity or leading token of an entity
# I => subsequent token of an entity

Token dictionaries for the last 3 words of instance 0
{'text': 'DORNBIRN', 'start': 0, 'end': 8, 'id': 0, 'ws': True, 'label': 'B-ORT'}
{'text': 'In', 'start': 9, 'end': 11, 'id': 1, 'ws': True, 'label': 'O'}
{'text': 'der', 'start': 12, 'end': 15, 'id': 2, 'ws': True, 'label': 'O'}
Token dictionaries for the last 3 words of instance 1
{'text': 'FELDKIRCH', 'start': 0, 'end': 9, 'id': 0, 'ws': True, 'label': 'B-ORT'}
{'text': 'Im', 'start': 10, 'end': 12, 'id': 1, 'ws': True, 'label': 'O'}
{'text': 'Altenreuteweg', 'start': 13, 'end': 26, 'id': 2, 'ws': True, 'label': 'B-STRASSE'}


In [4]:
# use updated list of instances ["token"] and added ["label"] fields to create a list where each instance is ...
# ... represented by a list of (token, label) pairs
sents=[] 
for annot_i in annot:                  # loop over instances
    toks = annot_i['tokens']           # get tokens list for instance i
    train_sentence = []
    for tok in toks:                   # loop over token dicts
        if 'label' in tok:             # only if the current token has been labelled, ...
            token_element = (tok['text'], tok['label']) # ... create a "text", "label" pair for this token ...
            train_sentence.append(token_element)        # ... and append it to the list
    sents.append(train_sentence) # append the list for that instances to the list for all instances / sentences

# list of lists of pairs (sets): outer list contains instances and inner list contains (token, label) pairs
sents

[[('DORNBIRN', 'B-ORT'),
  ('In', 'O'),
  ('der', 'O'),
  ('Schulgasse', 'B-STRASSE'),
  ('in', 'O'),
  ('Dornbirn', 'B-ORT'),
  ('hat', 'O'),
  ('eine', 'O'),
  ('71,93', 'B-FLAECHE'),
  ('Quadratmeter', 'O'),
  ('große', 'O'),
  ('Wohnung', 'B-IMMO_TYP'),
  ('für', 'O'),
  ('einen', 'O'),
  ('Quadratmeterpreis', 'O'),
  ('von', 'O'),
  ('5533,71', 'B-GESAMTPREIS'),
  ('Euro', 'O'),
  ('den', 'O'),
  ('Besitzer', 'O'),
  ('gewechselt', 'O'),
  ('.', 'O'),
  ('Dieser', 'O'),
  ('beinhaltet', 'O'),
  ('auch', 'O'),
  ('einen', 'O'),
  ('Pkw-Abstellplatz', 'O'),
  ('.', 'O'),
  ('Käufer', 'O'),
  ('der', 'O'),
  ('Wohnung', 'O'),
  ('mit', 'O'),
  ('9,86', 'B-TERRASSENGROESSE'),
  ('Quadratmetern', 'O'),
  ('Terrasse', 'O'),
  ('ist', 'O'),
  ('die', 'O'),
  ('ValLiLean', 'B-KAEUFER'),
  ('Beteiligungs-', 'I-KAEUFER'),
  ('und', 'I-KAEUFER'),
  ('Immobilienverwaltungs', 'I-KAEUFER'),
  ('GmbH.', 'I-KAEUFER'),
  ('Beim', 'O'),
  ('Verkäufer', 'O'),
  ('handelt', 'O'),
  ('es', 'O'),
  ('s

In [5]:
def word2features(sent, i): # receive instance and the index i for the i-th token of the instance
    word = sent[i][0]       # i-th token ("sent" is a list of (token, label) pairs; the label is not used, here)
    
    # dictionary of features for the i-th token
    features = {
        'bias': 1.0,                      # a different bias could be computed here (static 1 is useless)
        'word.lower()': word.lower(),     # token in lowercase
        'word[-3:]': word[-3:],           # last 3 letters
        'word[-2:]': word[-2:],           # last 2 letters
        'word.isupper()': word.isupper(), # True if uppercase else False
        'word.istitle()': word.istitle(), # True if title else False, see...
        # ... https://www.w3schools.com/python/trypython.asp?filename=demo_ref_string_istitle2
        'word.isdigit()': word.isdigit()  # True if digit else False
    }
    if i > 0:
        # add features for the previous token
        word_minus_1 = sent[i-1][0]
        features.update({
            '-1:word.lower()': word_minus_1.lower(),
            '-1:word.istitle()': word_minus_1.istitle(),
            '-1:word.isupper()': word_minus_1.isupper(),
        })
    else: # the beginning of the sequence
        features['BOS'] = True

    if i < len(sent)-1:
        # add features for the next token
        word1 = sent[i+1][0]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
        })
    else: # the end of the sequence
        features['EOS'] = True

    return features


def sent2features(sent):                                      # pass list of (token, label) pairs
    return [word2features(sent, i) for i in range(len(sent))] # pass list of (token, label) pairs and the index ...
    # ... for a position in that list => obtain dict with token features => return list of such feature dicts

# obtain list of dicts with features for the corresponding tokens
sent2features(sents[0])[:2] 

[{'bias': 1.0,
  'word.lower()': 'dornbirn',
  'word[-3:]': 'IRN',
  'word[-2:]': 'RN',
  'word.isupper()': True,
  'word.istitle()': False,
  'word.isdigit()': False,
  'BOS': True,
  '+1:word.lower()': 'in',
  '+1:word.istitle()': True,
  '+1:word.isupper()': False},
 {'bias': 1.0,
  'word.lower()': 'in',
  'word[-3:]': 'In',
  'word[-2:]': 'In',
  'word.isupper()': False,
  'word.istitle()': True,
  'word.isdigit()': False,
  '-1:word.lower()': 'dornbirn',
  '-1:word.istitle()': False,
  '-1:word.isupper()': True,
  '+1:word.lower()': 'der',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False}]

In [6]:
def sent2labels(sent):   # pass one instance (=list of (token, label) pairs) => return list of labels
    return [label for token, label in sent]

X = [sent2features(s) for s in sents] # list of instances, each a list of tokens represented by a feature dict ...
                                      # ... => list of dictionaries
y = [sent2labels(s) for s in sents]   # list of instances, each a list of tokens represented by a label
print(len(X))
[print(X[0][i]) for i in range(4)]
print("")
print(len(y))
[print(y[0][i]) for i in range(4)]
print("")

140
{'bias': 1.0, 'word.lower()': 'dornbirn', 'word[-3:]': 'IRN', 'word[-2:]': 'RN', 'word.isupper()': True, 'word.istitle()': False, 'word.isdigit()': False, 'BOS': True, '+1:word.lower()': 'in', '+1:word.istitle()': True, '+1:word.isupper()': False}
{'bias': 1.0, 'word.lower()': 'in', 'word[-3:]': 'In', 'word[-2:]': 'In', 'word.isupper()': False, 'word.istitle()': True, 'word.isdigit()': False, '-1:word.lower()': 'dornbirn', '-1:word.istitle()': False, '-1:word.isupper()': True, '+1:word.lower()': 'der', '+1:word.istitle()': False, '+1:word.isupper()': False}
{'bias': 1.0, 'word.lower()': 'der', 'word[-3:]': 'der', 'word[-2:]': 'er', 'word.isupper()': False, 'word.istitle()': False, 'word.isdigit()': False, '-1:word.lower()': 'in', '-1:word.istitle()': True, '-1:word.isupper()': False, '+1:word.lower()': 'schulgasse', '+1:word.istitle()': True, '+1:word.isupper()': False}
{'bias': 1.0, 'word.lower()': 'schulgasse', 'word[-3:]': 'sse', 'word[-2:]': 'se', 'word.isupper()': False, 'word

In [7]:
train_ratio = 0.75
train_test_split = round(0.75*len(X) - 0.5) # -0.5 => floor
train_test_split
X_train = X[:train_test_split]
y_train = y[:train_test_split]
X_test = X[train_test_split:]
y_test = y[train_test_split:]
len(X_train), len(y_train), len(X_test), len(y_test)

(104, 104, 36, 36)

In [8]:
%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 4.05 µs


/Users/matthias/opt/anaconda3/envs/nlu/lib/python3.8/site-packages/sklearn/base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [9]:
eli5.show_weights(crf, top=30)

From \ To,O,B-DATUM_VERBUECHERUNG,I-DATUM_VERBUECHERUNG,B-DATUM_VERTRAG,I-DATUM_VERTRAG,B-FLAECHE,B-GESAMTPREIS,I-GESAMTPREIS,B-IMMO_TYP,I-IMMO_TYP,B-KAEUFER,I-KAEUFER,B-ORT,I-ORT,B-QMPREIS,B-STRASSE,I-STRASSE,B-TERRASSENGROESSE,B-VERKAEUFER,I-VERKAEUFER
O,2.494,1.673,-1.694,1.814,-2.009,2.083,1.403,-1.036,2.027,-3.114,1.731,-2.081,0.645,-1.113,1.403,1.961,-2.218,0.753,1.89,-2.399
B-DATUM_VERBUECHERUNG,-1.524,-0.465,4.77,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.001
I-DATUM_VERBUECHERUNG,-0.591,0.0,1.132,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-DATUM_VERTRAG,-0.824,0.0,0.0,-0.532,4.661,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-DATUM_VERTRAG,0.147,0.0,0.0,0.0,3.853,0.0,0.0,0.0,0.0,0.0,0.0,-0.031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.17
B-FLAECHE,1.55,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-GESAMTPREIS,0.972,0.0,0.0,0.0,-0.024,0.0,0.0,2.234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.076
I-GESAMTPREIS,0.417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-IMMO_TYP,0.288,0.0,-0.237,0.0,-0.145,0.0,0.0,0.0,-0.237,3.264,0.0,-0.347,0.0,0.0,0.0,0.313,-0.032,0.0,0.0,-0.425
I-IMMO_TYP,-0.053,0.0,0.0,0.0,0.0,-0.133,0.0,0.0,-0.013,3.83,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.025,0.0,0.0


In [10]:
labels = list(crf.classes_)
labels.remove('O')
labels

['B-ORT',
 'B-STRASSE',
 'B-FLAECHE',
 'B-IMMO_TYP',
 'B-GESAMTPREIS',
 'B-TERRASSENGROESSE',
 'B-KAEUFER',
 'I-KAEUFER',
 'B-VERKAEUFER',
 'I-VERKAEUFER',
 'B-DATUM_VERTRAG',
 'I-DATUM_VERTRAG',
 'B-DATUM_VERBUECHERUNG',
 'I-DATUM_VERBUECHERUNG',
 'B-QMPREIS',
 'I-IMMO_TYP',
 'I-STRASSE',
 'I-GESAMTPREIS',
 'I-ORT']

In [11]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=labels)

/Users/matthias/opt/anaconda3/envs/nlu/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(


0.9108921781594944

In [12]:
# group B and I results
sorted_labels = sorted(labels, key=lambda name: (name[1:], name[0]))
print(metrics.flat_classification_report(y_test, y_pred, labels=sorted_labels, digits=3))

/Users/matthias/opt/anaconda3/envs/nlu/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass labels=['B-DATUM_VERBUECHERUNG', 'I-DATUM_VERBUECHERUNG', 'B-DATUM_VERTRAG', 'I-DATUM_VERTRAG', 'B-FLAECHE', 'B-GESAMTPREIS', 'I-GESAMTPREIS', 'B-IMMO_TYP', 'I-IMMO_TYP', 'B-KAEUFER', 'I-KAEUFER', 'B-ORT', 'I-ORT', 'B-QMPREIS', 'B-STRASSE', 'I-STRASSE', 'B-TERRASSENGROESSE', 'B-VERKAEUFER', 'I-VERKAEUFER'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/matthias/opt/anaconda3/envs/nlu/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matthias/opt/anaconda3/envs/nlu/lib/python3.8/site-packages/sklearn/

                       precision    recall  f1-score   support

B-DATUM_VERBUECHERUNG      1.000     1.000     1.000        15
I-DATUM_VERBUECHERUNG      1.000     1.000     1.000        15
      B-DATUM_VERTRAG      0.933     0.933     0.933        15
      I-DATUM_VERTRAG      0.931     0.964     0.947        28
            B-FLAECHE      0.944     1.000     0.971        17
        B-GESAMTPREIS      1.000     0.591     0.743        22
        I-GESAMTPREIS      0.000     0.000     0.000         0
           B-IMMO_TYP      0.889     0.727     0.800        22
           I-IMMO_TYP      0.000     0.000     0.000         1
            B-KAEUFER      0.875     0.933     0.903        15
            I-KAEUFER      0.913     1.000     0.955        21
                B-ORT      0.957     0.898     0.926        49
                I-ORT      1.000     0.929     0.963        14
            B-QMPREIS      1.000     1.000     1.000        11
            B-STRASSE      0.917     0.786     0.846  